In [ ]:
#Processing initial data
import pandas as pd

#Modify this path for the file.
df = pd.read_csv('D:\\NGSIM数据处理\\NGSIM.csv', low_memory=False)
df = df[df['Location'].isin(['peachtree', 'lankershim'])]
df.to_csv('D:\\NGSIM数据处理\\processed_ngsim_data.csv', index=False)


In [134]:
import pandas as pd
import numpy as np
#Modify this path for the file.
filename = 'D:\\NGSIM数据处理\\processed_ngsim_data.csv'
num_lines = 200000

ngsim_df = pd.read_csv(filename, nrows=num_lines)

# 接下来可以对读取的数据进行处理
# 例如，打印前几行数据
print(ngsim_df.head())


   Vehicle_ID  Frame_ID Total_Frames    Global_Time  Local_X  Local_Y  \
0         879      6336          811  1118936313500  -24.046  513.096   
1        1503      8609        1,130  1118937560800   -6.564  996.874   
2         633      3318          275  1118937031700  -51.513  399.214   
3        1528      8866          404  1118937586500  -24.053  452.041   
4        1350     10267          925  1118936706600  -14.416  520.815   

      Global_X     Global_Y  v_length  v_Width  ...  D_Zone  Int_ID  \
0  6452106.035  1873271.842      15.0      7.0  ...   201.0     0.0   
1  6452313.510  1873714.849      16.2      7.4  ...   203.0     0.0   
2  6452031.973  1873195.646      12.5      7.0  ...   205.0     2.0   
3  6452083.184  1873225.318      19.0      8.0  ...   201.0     2.0   
4  6452118.062  1873274.613      18.0      7.0  ...   203.0     0.0   

   Section_ID Direction  Movement  Preceding  Following  Space_Headway  \
0         3.0       4.0       1.0        869        881     

C:\Users\29602\AppData\Local\Temp\ipykernel_17740\1335631656.py:6: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  ngsim_df = pd.read_csv(filename, nrows=num_lines)


In [135]:
# Fill missing values
ngsim_df = ngsim_df.fillna(0)

# Convert speed from feet per second to kilometers per hour
ngsim_df['v_Vel'] = ngsim_df['v_Vel'] * 1.09728


In [136]:
# Calculate the position of each vehicle at each time point in meters
print(f"Input rows: {len(ngsim_df)}")

# Calculate the position of each vehicle at each time point in meters
ngsim_df['position'] = (ngsim_df['Global_X']**2 + ngsim_df['Global_Y']**2)**0.5 / 0.3048  # convert feet to meters

# Divide each 100 meters into a section
ngsim_df['Section_ID'] = (ngsim_df['position'] // 100).astype(int)

# Calculate the maximum speed
max_speed = ngsim_df.groupby('Section_ID')['v_Vel'].max()

# Add the maximum speed to the original dataframe
ngsim_df = ngsim_df.set_index('Section_ID')
ngsim_df['max_speed'] = max_speed
ngsim_df = ngsim_df.reset_index()

# Calculate speed loss
ngsim_df['speed_loss'] = ngsim_df['max_speed'] - ngsim_df['v_Vel']

# Calculate time loss
ngsim_df['time_loss'] = (ngsim_df['speed_loss'] / ngsim_df['max_speed']) * 45

# Convert Global_Time into a 45-second cycle
ngsim_df['Global_Time'] = ngsim_df.groupby('Section_ID')['Global_Time'].transform(lambda x: x - x.min())

# Convert Global_Time into a 45-second cycle
ngsim_df['Global_Time'] = (ngsim_df['Global_Time'] // 45000) * 45000

# For each vehicle at each time point, calculate its change in the Local_Y direction
ngsim_df['Local_Y'] = ngsim_df['Local_Y'].astype(str).str.replace(',', '').astype(float)

# For each vehicle at each time point, calculate its change in the Local_Y direction
ngsim_df['y_diff'] = ngsim_df.sort_values(by='Global_Time').groupby('Vehicle_ID')['Local_Y'].diff()

# Determine whether each vehicle enters or leaves the section at each time point based on the value of y_diff
ngsim_df['enter'] = (ngsim_df['y_diff'] > 0).astype(int)
ngsim_df['leave'] = (ngsim_df['y_diff'] < 0).astype(int)

# Deduplicate records for each vehicle within each 45-second cycle
ngsim_df = ngsim_df.sort_values('Global_Time').groupby(['Vehicle_ID', 'Section_ID', 'Global_Time']).first().reset_index()

# Convert speed from kilometers per hour to meters per second
ngsim_df['v_Vel_m_s'] = ngsim_df['v_Vel'] * 1000 / 3600

# Filter out records with speeds lower than 5.0
ngsim_df = ngsim_df[ngsim_df['v_Vel_m_s'] >= 5.0]

# Drop rows with missing speed values
ngsim_df = ngsim_df.dropna(subset=['v_Vel_m_s'])

# Calculate the time spent by each vehicle on each section in seconds
ngsim_df['time_spent'] = 100 / ngsim_df['v_Vel_m_s']

# Perform statistical calculations
result_df = ngsim_df.groupby(['Section_ID', 'Global_Time']).agg(
    meanSpeed=('v_Vel', 'mean'),
    meanTimeLoss=('time_loss', 'mean'),
    nVehLeft=('leave', 'sum'),
    nVehsSeen=('Vehicle_ID', 'count'),
    nVehEntered=('enter', 'sum'),
    sampledSeconds=('time_spent', 'sum')
).reset_index()

# Calculate end time
result_df.columns = ['Section_ID', 'begin', 'meanSpeed', 'meanTimeLoss', 'nVehLeft', 'nVehsSeen', 'nVehEntered', 'sampledSeconds']

# Calculate end time
result_df['begin'] = result_df['begin'] / 1000
result_df['end'] = result_df['begin'] + 45
# Insert 'end' column to the right of 'begin' column
end_col = result_df.pop('end')
result_df.insert(result_df.columns.get_loc('begin') + 1, 'end', end_col)

# Find the maximum value of meanTimeLoss
max_time_loss = result_df['meanTimeLoss'].max()

# Divide each meanTimeLoss value by the maximum value and subtract it from 1
result_df['meanTimeLoss'] = 1 - (result_df['meanTimeLoss'] / max_time_loss)
row['meanTimeLoss'] = f"{mean_time_loss:.2f}"
# Keep it as the original format of model input
# Print the number of rows in the processed data
print(f"Output rows: {len(result_df)}")

# Output the results
result_df.to_csv('D:\\NGSIM数据处理\\Test_Set_NGSIM_speed.csv', index=False)


Input rows: 200000
Output rows: 2411


In [137]:
import pandas as pd

# Read file
df = pd.read_csv('D:\\NGSIM数据处理\\Test_Set_NGSIM_speed.csv')

# Modify the file to the input of model
df = df.drop(columns=['Section_ID'])
df = df.rename(columns={'nVehsSeen': 'nVehSeen'})
df = df[['begin', 'end', 'sampledSeconds', 'nVehEntered', 'nVehLeft', 'nVehSeen', 'meanSpeed', 'meanTimeLoss']]

# Save the new dataframe as a file
df.to_csv('D:\\NGSIM数据处理\\NGSIM_Test_Set.csv', index=False)
